### Class 5 - Uncertainty in Supply Chain Networks
RSM-8423, Winter 2023

### Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp

from scipy import stats

# Initialize seaborn (for plotting)
sns.set()

### Loading and inspecting data

The same as in Class 4

In [2]:
# Depots
dfDepots = pd.read_csv("depots.csv", index_col=0)
dfDepots

,Cost,Capacity
Depot,,
1,14754,120
2,5242,75


In [3]:
# Demand Zones
dfZones = pd.read_csv("zones.csv")
dfZones

,Demand Zone,Scenario,Demand,Probability
0,1,1,10,0.4
1,1,2,15,0.5
2,1,3,100,0.1


In [4]:
# Zone-depot costs
dfZoneDepot = pd.read_csv("zone_depot_costs.csv")
dfZoneDepot

,Demand Zone,Depot,Cost
0,1,1,1998
1,1,2,400


### Sets

In [5]:
# Depots
depots = list(dfDepots.index)
numdepots = len(depots)

# Demand zones
zones = list(dfZones["Demand Zone"].unique())
numzones = len(zones)

# Scenarios
scenarios = dfZones["Scenario"].unique()
numscenarios = len(scenarios)

### Parameters

In [6]:
# Probability of each scenario (dictionary)
probscenario = {}
for s in scenarios:
    probscenario[s] = float(dfZones[dfZones["Scenario"] == s].iloc[0]["Probability"])
    
# Demand in each zone and scenario (dictionary)
zonedemand = {}
for j in zones:
    for s in scenarios:
        zonedemand[(j,s)] = float(dfZones[(dfZones["Demand Zone"] == j) & (dfZones["Scenario"] == s)]["Demand"])

# Depot capacities (dictionary)
depotcapacity = {}
for i in depots:
    depotcapacity[i] = int(dfDepots.loc[i]["Capacity"])

# Depot costs (dictionary)
depotcost = {}
for i in depots:
    depotcost[i] = float(dfDepots.loc[i]["Cost"])
    
# Depot-zone costs (dictionary)
depotzonecost = {}
for i in depots:
    for j in zones:
        depotzonecost[(i,j)] = float(dfZoneDepot[(dfZoneDepot["Depot"] == i)&(dfZoneDepot["Demand Zone"] == j)]["Cost"])

### Variables

In [7]:
# Variables: if a depot is opened/allocated
yvar = pulp.LpVariable.dict("y", depots, cat=pulp.LpBinary)

# Variables: amount of demand from each zone allocated to depot, per scenario
xvar = pulp.LpVariable.dict("x", (depots, zones, scenarios), lowBound=0.0, cat=pulp.LpContinuous)

### Model initialization


In [8]:
# Initialize model and objective sense
locationModel = pulp.LpProblem(name="LocationModel", sense=pulp.LpMinimize)

### Constraints

In [9]:
# Contraint: demand must be satisfied in all scenarios
for j in zones:
    for s in scenarios:    
        locationModel += pulp.lpSum( [xvar[(i,j,s)] for i in depots] ) == zonedemand[(j,s)]

# Constraint: depot capacities must be observed in all scenarios
for i in depots:    
    for s in scenarios:    
        locationModel += pulp.lpSum( [xvar[(i,j,s)] for j in zones] ) <= depotcapacity[i] * yvar[i]

### Objective function

In [10]:
# Objective function

# --- depot allocation costs
obj = pulp.lpSum([ depotcost[i] * yvar[i] for i in depots])

# --- package flow costs
for s in scenarios:
    obj += probscenario[s] * pulp.lpSum( [ depotzonecost[(i,j)] * xvar[(i,j,s)] for i in depots for j in zones ] )

# add objective to model
locationModel += obj

### Solution process

In [11]:
# Write LP to file (optional, but often good to inspect model and find errors)
locationModel.writeLP("locationModel.lp")

# Solve model
locationModel.solve()
print("Status:", pulp.LpStatus[locationModel.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/cireandr/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/p6/k9kpfkxx7h78ybc6p8j_mp2m0000gp/T/794b791846354e339b24520a734a6157-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/p6/k9kpfkxx7h78ybc6p8j_mp2m0000gp/T/794b791846354e339b24520a734a6157-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 45 RHS
At line 55 BOUNDS
At line 58 ENDATA
Problem MODEL has 9 rows, 8 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20910.8 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (1 integer (1 of which binary)) and 6 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 32591
Cbc0038I Relaxing continuous gives 32591
Cbc0038I Before mini branch and

In [12]:
# Total cost
totalCost = pulp.value(locationModel.objective)
print("Total cost: " + str(totalCost))

Total cost: 32591.0


In [13]:
# Print solution (you can add to the dataframe if needed)
for i in depots:
    if yvar[i].varValue >= 1.0:
        print("Depot " + str(i) + " is opened")
        
        for s in scenarios:
            print("\tScenario " + str(s) + ":")
            for j in zones:
                if xvar[(i,j,s)].varValue > 0.0:
                    print("\t\tServes zone " + str(j) + " - with capacity " + str(xvar[(i,j,s)].varValue))

Depot 1 is opened
	Scenario 1:
	Scenario 2:
	Scenario 3:
		Serves zone 1 - with capacity 25.0
Depot 2 is opened
	Scenario 1:
		Serves zone 1 - with capacity 10.0
	Scenario 2:
		Serves zone 1 - with capacity 15.0
	Scenario 3:
		Serves zone 1 - with capacity 75.0
